In [1]:
!pip install scikit-fuzzy

   ---------------------------------------- 0.0/920.8 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/920.8 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------- ----- 786.4/920.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 920.8/920.8 kB 870.1 kB/s  0:00:00


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# ------------------------------------------------------
# 1. Definición de las variables de entrada y salida
# ------------------------------------------------------

# Entradas:
aceleracion = ctrl.Antecedent(np.arange(0, 1.1, 0.01), 'aceleracion')    # g (0–1g)
frecuencia  = ctrl.Antecedent(np.arange(0, 20.1, 0.1), 'frecuencia')    # Hz
duracion    = ctrl.Antecedent(np.arange(0, 60.1, 1), 'duracion')        # s (0–60) 
distancia   = ctrl.Antecedent(np.arange(0, 601, 1), 'distancia')        # km (0–600)
snr         = ctrl.Antecedent(np.arange(0, 30.1, 0.1), 'snr')           # dB

# Salida:
amenaza = ctrl.Consequent(np.arange(0, 11, 1), 'amenaza')              # 0–10 escala de riesgo

# ------------------------------------------------------
# 2. Definición de las funciones de pertenencia
# ------------------------------------------------------

## Distancia (km)
distancia['muy_cercana'] = fuzz.trapmf(distancia.universe, [0, 0, 50, 150]) 
distancia['cercana'] = fuzz.trimf(distancia.universe, [100, 200, 300]) 
distancia['moderada'] = fuzz.trimf(distancia.universe, [250, 375, 500])
distancia['lejana'] = fuzz.trapmf(distancia.universe, [450, 550, 600, 600])

# Aceleración (g)
aceleracion['baja']  = fuzz.trimf(aceleracion.universe, [0,   0,   0.3])
aceleracion['media'] = fuzz.trimf(aceleracion.universe, [0.2, 0.5, 0.8])
aceleracion['alta']  = fuzz.trimf(aceleracion.universe, [0.6, 1,   1])

# Frecuencia (Hz)
frecuencia['baja']  = fuzz.trapmf(frecuencia.universe,  [0, 0, 2, 5])
frecuencia['media'] = fuzz.trapmf(frecuencia.universe,  [3, 6, 10, 13])
frecuencia['alta']  = fuzz.trapmf(frecuencia.universe,  [11, 15, 20, 20])

# Duración (s)
duracion['corta'] = fuzz.trimf(duracion.universe, [0, 0, 10])
duracion['media'] = fuzz.trimf(duracion.universe, [5, 20, 35])
duracion['larga'] = fuzz.trimf(duracion.universe, [30, 60, 60])

# Signal-to-Noise Ratio (dB)
snr['pobre']      = fuzz.trapmf(snr.universe, [0,  0,  5,  10])
snr['aceptable']  = fuzz.trapmf(snr.universe, [8,  12, 18, 22])
snr['buena']      = fuzz.trapmf(snr.universe, [20, 25, 30, 30])

# Amenaza sísmica (0–10)
amenaza['sin_actividad'] = fuzz.trimf(amenaza.universe, [0,  0,  2])
amenaza['microtemblor']  = fuzz.trimf(amenaza.universe, [1,  3,  5])
amenaza['leve']          = fuzz.trimf(amenaza.universe, [4,  5,  6])
amenaza['moderado']      = fuzz.trimf(amenaza.universe, [5,  7,  8])
amenaza['fuerte']        = fuzz.trimf(amenaza.universe, [7,  9,  10])
amenaza['destructivo']   = fuzz.trimf(amenaza.universe, [9,  10, 10])

# ------------------------------------------------------
# 3. Definición de las reglas difusas (Lógica principal con DISTANCIA)
# ------------------------------------------------------

rules = []

# === CASO 1: PEQUEÑOS/RUIDO ===
# Si la aceleración es baja Y la SNR es pobre, no hay amenaza (sin actividad o ruido)
rules.append(ctrl.Rule(aceleracion['baja'] & snr['pobre'], amenaza['sin_actividad']))

# Si la aceleración es baja Y la distancia es lejana, la amenaza es mínima.
rules.append(ctrl.Rule(aceleracion['baja'] & distancia['lejana'], amenaza['microtemblor']))

# === CASO 2: MODERADOS/LEVES (Distancia y Frecuencia) ===
# Si la aceleración es media Y la frecuencia es alta (típico de sismos pequeños) Y la distancia es lejana, es leve.
rules.append(ctrl.Rule(aceleracion['media'] & frecuencia['alta'] & distancia['lejana'], amenaza['leve']))

# Si la aceleración es media Y la frecuencia es media Y la distancia es muy_cercana, se eleva a moderado.
rules.append(ctrl.Rule(aceleracion['media'] & frecuencia['media'] & distancia['muy_cercana'], amenaza['moderado']))

# === CASO 3: FUERTES/DESTRUCTIVOS (Duración y Proximidad) ===
# La regla más crítica: gran aceleración, baja frecuencia (gran onda) y larga duración, cerca del borde, buena SNR.
rules.append(ctrl.Rule(
    aceleracion['alta'] & frecuencia['baja'] & duracion['larga'] & distancia['muy_cercana'] & snr['buena'], 
    amenaza['destructivo']
))

# Caso Fuerte, pero no en el epicentro (distancia cercana/moderada)
rules.append(ctrl.Rule(
    aceleracion['alta'] & frecuencia['baja'] & duracion['media'] & (distancia['cercana'] | distancia['moderada']), 
    amenaza['fuerte']
))

# Caso Moderado/Leve, pero confiable (SNR)
rules.append(ctrl.Rule(
    aceleracion['media'] & snr['buena'] & distancia['cercana'], 
    amenaza['moderado']
))

# === CASO DE CONFIANZA BAJA EN DATOS FÍSICOS (SNR Pobre) ===
# Si los datos físicos indican algo fuerte, pero la SNR es pobre, se reduce la confianza.
rules.append(ctrl.Rule(
    aceleracion['alta'] & snr['pobre'],
    amenaza['microtemblor'] 
))

# ------------------------------------------------------
# 4. Creación del sistema de control difuso
# ------------------------------------------------------
control_sismo = ctrl.ControlSystem(rules)
simulador = ctrl.ControlSystemSimulation(control_sismo)

# ------------------------------------------------------
# 5. Ejemplo de simulación
# ------------------------------------------------------

def ejecutar_simulacion():
    """Solicita entradas al usuario y calcula la amenaza sísmica."""
    try:
        # Pide las entradas de forma segura
        valor_acel = float(input("Ingrese la aceleración del terreno (g, 0–1): "))
        valor_freq = float(input("Ingrese la frecuencia dominante (Hz, 0–20): "))
        valor_dur  = float(input("Ingrese la duración del evento (s, 0–60): "))
        valor_dist = float(input("Ingrese la distancia al borde de placa tectónica (km, 0–600): "))
        valor_snr  = float(input("Ingrese la SNR (dB, 0–30): "))
        
        # Asignar inputs al simulador
        simulador.input['aceleracion'] = valor_acel
        simulador.input['frecuencia']  = valor_freq
        simulador.input['duracion']    = valor_dur
        simulador.input['distancia']   = valor_dist
        simulador.input['snr']         = valor_snr

        # Calcular
        simulador.compute()
        nivel_amenaza = simulador.output['amenaza']

        # Mostrar resultados
        print(f"\n🔍 Nivel de amenaza sísmica (defuzzificado): {nivel_amenaza:.2f} / 10")
        
        if nivel_amenaza < 2.5:
            print("➡️ Clasificación: Sin actividad sísmica o ruido.")
        elif nivel_amenaza < 4.5:
            print("➡️ Clasificación: Microtemblor / Sismo Leve.")
        elif nivel_amenaza < 7.5:
            print("➡️ Clasificación: Terremoto Moderado.")
        elif nivel_amenaza < 9.0:
            print("➡️ Clasificación: Terremoto Fuerte.")
        else:
            print("➡️ Clasificación: Terremoto Destructivo.")
            
    except ValueError:
        print("\n❌ Error: Asegúrate de ingresar valores numéricos válidos en cada campo.")
    except Exception as e:
        print(f"\n❌ Ha ocurrido un error inesperado: {e}")

# Ejecutar el sistema
if __name__ == '__main__':
    ejecutar_simulacion()


🔍 Nivel de amenaza sísmica: 0.67 / 10
